In [1]:
from importlib import reload 
import sys
import pandas as pd
import numpy as np
from scipy.stats import pearsonr, spearmanr, kendalltau
import plotly.express as px
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.spatial.distance import braycurtis
from scipy.special import softmax
import seaborn as sns
# # adding Folder_2 to the system path
# sys.path.insert(0, r'C:\Users\tomer\Desktop\BSc\year3\sem B\workshop_microbiome\code')
# # sys.path.insert(0, r'C:\Users\yuvald\Documents\Uni\סמסטר ב\workshop_microbiome\code')

from imports import  *

import social_model_class
reload(social_model_class)


: 

In [2]:
data_path = r"train_data.csv"
metadata_path = r"train_metadata.csv"

$$\Delta_t  \space windowing$$

In [ ]:
model = social_model_class.superModel(data_path, metadata_path)


: 

In [ ]:
model.fit()

In [ ]:
model.fit()

In [ ]:
model.baboons[0].fit(np.array([1/3]*61))

In [5]:
metadata_df = pd.read_csv(metadata_path)
data_df = pd.read_csv(data_path)

metadata_df["collection_date"] = pd.to_datetime(metadata_df["collection_date"])

In [6]:
metadata_df.set_index('sample', inplace=True)
data_df.set_index('sample', inplace=True)

In [ ]:
model = social_model_class.superModel(data_path,   metadata_path)

In [10]:
metadata_df["collection_date_datetime"] = pd.to_datetime(metadata_df["collection_date"])

In [16]:
metadata_df.drop_duplicates(subset=["baboon_id", "collection_date"], inplace=True)

In [31]:
time_samples_df = metadata_df[["social_group", "collection_date_datetime"]].copy()

In [32]:
time_samples_df["delta_t_cnt"] = 0

In [ ]:
for group in time_samples_df["social_group"].unique():
    for t in time_samples_df["collection_date_datetime"][time_samples_df["social_group"] == group]:
        count = len(time_samples_df[(time_samples_df["social_group"]==group) & (abs(time_samples_df["collection_date_datetime"]-t).dt.days<=10)])
        time_samples_df["delta_t_cnt"][(time_samples_df["social_group"]==group) & (time_samples_df["collection_date_datetime"]==t)] = count
time_samples_df.drop_duplicates(inplace=True)

In [ ]:
for group in time_samples_df["social_group"].unique():
    plt.plot(sorted(time_samples_df["collection_date_datetime"][time_samples_df["social_group"]==group]), time_samples_df["delta_t_cnt"][time_samples_df["social_group"]==group], label=group)


In [ ]:
for t in time_samples_df.index:
    delta_t_datetimes = [i for i in range(t-7,t+7)]
    time_samples_df[""]

In [ ]:
time_samples_df.loc[pd.to_datetime('2000-05-19')]

In [ ]:
resampled_df